In [1]:
from utils.config_loader import load_config, assign_config
from processing.climatology import process_climatology
from processing.preprocessing import preprocess_data
from processing.normalize import normalize_data
from model.train import train_model
from model.inference import infer
from model.metrics import calculate_rmse
from visualization.plots import plot_heatmaps

config = load_config("./config/params.yaml")

Login to wandb and init experiment

In [3]:
import wandb
import os

os.environ['WANDB_NOTEBOOK_NAME'] = "pipeline.ipynb"
wandb.login()

run = wandb.init(
    project="WindViVit2",
    
    notes="Real data",
    tags=["45 years data", "BERT Base"]
)

wandb_config = wandb.config

assign_config(wandb_config, config)

wandb: Currently logged in as: samuelreyes. Use `wandb login --relogin` to force relogin


### Data preprocessing

In [2]:
process_climatology(config)

In [3]:
preprocess_data(config)

In [4]:
normalize_data(config)

### Training

In [4]:
train_model(config)

KeyboardInterrupt: 

### Evaluate model

In [ ]:
results = infer(config)

In [ ]:
rmse = calculate_rmse(config, results)
wandb.log({"upper_rmse": rmse["upper_rmse"], "surface_rmse": rmse["surface_rmse"]})
rmse

In [ ]:
fig = plot_heatmaps(config, results, 5)
wandb.log({"heatmap": fig})

In [5]:
run.finish()

LR,█▆▃▁
Step,▁▃▆█
LR,0.0001
Step,300
